dQ/dt=(kA(T2-T1))/d
T=mc/Q
Q=heat transfered
k=thermal conductivity
A=surface area
T1=first temp
T2=second temp
d=thickness
m=mass
c=specific heat

In [ ]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [ ]:
def make_system(conductivity, areaOuter, areaMiddle, areaInner, thicknessOuter, thicknessMiddle, thicknessInner, mass, specific):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(outer, middle, inner, oTemp, mTEmp, iTemp)
    conductivity=0.1
    areaOuter=0.0454
    areaMiddle=0.0454
    areaInner=0.0454
    thicknessOuter=0
    thicknessMiddle=0
    thicknessInner=0
    mass=2.33
    specific=2.02
    
    return System(init=init, k=k, SA=SA, d=d)

In [ ]:
def update_func(state, t, system):
    """Update the SIR model.
    
    state: State with variables S, I, R
    t: time step
    system: System with beta and gamma
    
    returns: State object
    """
    outer, middle, inner, oTemp, mTemp, iTemp = state
    unpack(system)

    dQdt1 = (conductivity*areaMiddle*(oTemp-mTemp))/thicknessMiddle
    dQdt2 = (conductivity*areaInner*(mTemp-iTemp))/thicknessInner
    dTdtinner = (mass*specific)/inner
    dTdtmiddle = (mass*specific)/middle
    dTdtouter = (mass*specific)/outer
    
    outer += -dQdt1
    middle += dQdt1 - dQdt2
    inner += dQdt2
    
    oTemp += dTdtmiddle
    mTemp += dTdtmiddle
    iTemp += dTdtinner
    
    
    return State(outer=outer, middle=middle inner=inner)